In [4]:
import cv2
import numpy as np
import os

In [5]:
os.getcwd()

'C:\\Users\\user\\Desktop\\pnh'

In [6]:
# Set numbers, size of crop
crop_n = 100
sel_p = 0.0  # ratio of fucused region
rand_n = int(crop_n * (1 - sel_p))
crop_wth = 64
crop_hgt = 64

# Get image list
img_list = os.listdir("labeling/img")
name_list = []
for img in img_list:
    name_list.append(os.path.splitext(img)[0])

# # Set focused reference coordinate
# # ref_xy = [[138, 105],
# #           [54, 460],
# #           [938, 154],
# #           [783, 716],
# #           [268, 727],
# #           [758, 765],
# #           [1295, 719],
# #           [685, 660],
# #           [978, 322]]

In [7]:
for k, img_name in enumerate(name_list):
    # Load bounding box coordinate & sample image
    box_coor = np.loadtxt("labeling/bbox/" + img_name + ".txt",
                          delimiter=' ')
    img_raw = cv2.imread("labeling/img/" + img_name + ".jpg")
    
    img_wth, img_hgt = img_raw.shape[1], img_raw.shape[0]
    
    # Convert bounding box coordinate of YOLO form to OpenCV form
    box_array = np.zeros_like(box_coor)[:, 1:]
    box_array[:, 2] = box_coor[:, 3] * img_wth
    box_array[:, 3] = box_coor[:, 4] * img_hgt
    box_array[:, 0] = box_coor[:, 1] * img_wth - box_array[:, 2] / 2
    box_array[:, 1] = box_coor[:, 2] * img_hgt - box_array[:, 3] / 2
    box_array = box_array.round().astype(int)
    
    box_array[:, 2:] = box_array[:, 2:] + 1
    box_array[:, :2] = box_array[:, :2] - 1
    
    #### Cropping Image to Patch ####    
    # Set reference coordinate of crop
    np.random.seed(42 + k * 10)
    crop_ref = np.array(
        [np.random.randint(low=0, high=img_wth - crop_wth, size=rand_n),
         np.random.randint(low=0, high=img_hgt - crop_hgt, size=rand_n)]
    ).T
    
    # # Set the focused reference range to crop
    # ref_sel = []
    # for i in range(len(ref_xy)):
    #     x_min = ref_xy[i][0] - crop_wth
    #     x_max = ref_xy[i][0] + crop_wth
    #     y_min = ref_xy[i][1] - crop_hgt
    #     y_max = ref_xy[i][1] + crop_hgt
    #     
    #     if x_min < 0:
    #         x_max = x_max - x_min
    #         x_min = 0
    #     if x_max > img_wth - crop_wth:
    #         x_min = x_min - (x_max - (img_wth - crop_wth))
    #         x_max = img_wth - crop_wth
    #     if y_min < 0:
    #         y_max = y_max - y_min
    #         y_min = 0
    #     if y_max > img_hgt - crop_hgt:
    #         y_min = y_min - (y_max - (img_hgt - crop_hgt))
    #         y_max = img_hgt - crop_hgt
    #     
    #     ref_s = [[x_min, x_max], [y_min, y_max]]
    #     ref_sel.append(ref_s)
    # 
    # 
    # sel_n = int((crop_n - rand_n) / len(ref_sel))
    # 
    # for i in range(len(ref_sel)):
    #     c_ref = np.array(
    #         [np.random.randint(low=ref_sel[i][0][0],
    #                            high=ref_sel[i][0][1],
    #                            size=sel_n),
    #          np.random.randint(low=ref_sel[i][1][0],
    #                            high=ref_sel[i][1][1],
    #                            size=sel_n)]
    #     ).T
    #     crop_ref = np.vstack((crop_ref, c_ref))
    
    
    # Change bounding box array format to ref. point & end point
    bboxes = box_array.copy()
    bboxes[:, 2] = bboxes[:, 0] + bboxes[:, 2]
    bboxes[:, 3] = bboxes[:, 1] + bboxes[:, 3]
    
    
    # Crop image & bounding box array
    for i, ref in enumerate(crop_ref):
        fname = img_name + "-" + format(ref[0], '04') + format(ref[1], '04')
        # Set crop coordinate
        end_cropy = ref[1] + crop_hgt
        end_cropx = ref[0] + crop_wth
        
        # Get cropped image
        img_crop = img_raw[ref[1]:end_cropy, ref[0]:end_cropx, :]
        
        # Filter bounding boxes out of the cropped image
        box_crop = bboxes[((bboxes[:, 2] > ref[0]) & (bboxes[:, 3] > ref[1])) & ((bboxes[:, 0] < end_cropx) & (bboxes[:, 1] < end_cropy))]
        
        # Change coordinate of the bounding boxes matching to the cropped image
        box_crop[:, [0, 2]] = box_crop[:, [0, 2]] - ref[0]
        box_crop[:, [1, 3]] = box_crop[:, [1, 3]] - ref[1]
        
        # Cut off extra regions of the bounding boxes
        box_crop[box_crop[:, 0] < 0, 0] = 0
        box_crop[box_crop[:, 1] < 0, 1] = 0
        box_crop[box_crop[:, 2] > crop_wth, 2] = crop_wth
        box_crop[box_crop[:, 3] > crop_hgt, 3] = crop_hgt
        
        # Change bounding box array format to YOLO style
        box_yolo = box_crop.copy().astype('float')
        box_yolo[:, 2] = box_yolo[:, 2] - box_yolo[:, 0]  # get box size
        box_yolo[:, 3] = box_yolo[:, 3] - box_yolo[:, 1]
        box_yolo[:, 0] = box_yolo[:, 0] + box_yolo[:, 2] / 2  # centering box coord.
        box_yolo[:, 1] = box_yolo[:, 1] + box_yolo[:, 3] / 2
        box_yolo[:, [0, 2]] = (box_yolo[:, [0, 2]] / crop_wth)
        box_yolo[:, [1, 3]] = (box_yolo[:, [1, 3]] / crop_hgt)
        box_yolo = np.hstack((np.zeros(box_yolo.shape[0])[:, np.newaxis], box_yolo))
        
        # Save cropped images & bounding boxes
        os.makedirs("data/raw/", exist_ok=True)
        cv2.imwrite("data/raw/" + fname + ".jpg", img_crop)
        np.savetxt("data/raw/" + fname + ".txt", box_yolo,
                   fmt='%i %0.6f %0.6f %0.6f %0.6f')